# Генерация ответа на русском языке (T-Lite)

### Установка требуемых модулей

In [17]:
!pip install -r requirements.txt

### Импорт модулей

In [16]:
from DataPrepare.docx_parser import DocxParser
from DataPrepare.vector_base import VectorBase
from DataPrepare.evaluation_metrics import EvaluationMetrics
from DataPrepare.generator import AdvancedGenerator

import pandas as pd

### Парсинг данных

In [2]:
dp = DocxParser(50)

In [3]:
documents = []

for doc in range(4):
    file_path = f"WordDocuments/sample{doc + 1}.docx"

    parsed_data = dp.parse_docx(file_path, f'sample{doc + 1}')
    documents.append(parsed_data)

### Создание векторной базы данных

In [4]:
db = VectorBase()
db.fit(documents)

Генерация контекста для запроса пользователя

In [5]:
query = "Name all types of screen readers"
search_results = db.search(query, top_k=10)

context = " ".join([t[0] for t in sorted([(result["text"], result["id"]) for result in search_results], key=lambda el: el[1])])

### Генерация ответа

In [6]:
generator = AdvancedGenerator()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [7]:
answer = generator.generate_answer(query, context)

print(query + '\n')
print(answer)

Name all types of screen readers

На основе предоставленного контекста, вот список упомянутых программ для чтения с экрана (screen readers):

1. JAWS
2. NVDA
3. Window-Eyes
4. System Access
5. VoiceOver

Если вы ищете более полный список, стоит отметить, что существуют и другие программы для чтения с экрана, такие как:

- TalkBack (для Android)
- Narrator (в Windows)
- ChromeVox (для Chrome OS)
- Orca (для Linux)

Эти программы помогают людям с нарушениями зрения или другими ограничениями воспринимать цифровой контент.


### Подсчет метрик

In [11]:
evaluator = EvaluationMetrics()

references = ['1. JAWS 2. NVDA 3. Window-Eyes 4. System Access 5. VoiceOver']
results = evaluator.evaluate(references,[answer])

In [12]:
print("ROUGE scores")
pd.DataFrame(results["rouge"])

ROUGE scores


,rouge-1,rouge-2,rouge-l
r,1.000000,1.000000,1.000000
p,0.171875,0.140845,0.171875
f,0.293333,0.246914,0.293333


In [13]:
print("BERTScore")
pd.DataFrame(results["bertscore"], index=['score']).T

BERTScore


,score
precision,0.509667
recall,0.890969
f1,0.648416
